In [ ]:

import heapq

def allocate_files(files, num_folders=10):
    total_size = sum(files)
    folder_capacity = 5192
    folders = [{'remaining': folder_capacity, 'index': i} for i in range(num_folders)]
    heap = [(-folder_capacity, i) for i in range(num_folders)]
    heapq.heapify(heap)
    
    allocations = {}
    file_details = []
    
    for idx, size in enumerate(sorted(files, reverse=True)):
        file_details.append((idx, size))
    
    for file_id, size in file_details:
        temp_heap = [(-f['remaining'], f['index']) for f in folders]
        heapq.heapify(temp_heap)
        selected = []
        used = set()
        
        while temp_heap and len(selected) < 3:
            rem_neg, idx = heapq.heappop(temp_heap)
            rem = -rem_neg
            if rem >= size and idx not in used:
                selected.append(idx)
                used.add(idx)
        
        if len(selected) >= 3:
            for idx in selected:
                folders[idx]['remaining'] -= size
            allocations[file_id] = {'split': False, 'locations': selected}
            continue
        
        s1 = size / 2
        s2 = size - s1
        if s1 > folder_capacity or s2 > folder_capacity:
            return None
        
        temp_heap_s1 = [(-f['remaining'], f['index']) for f in folders]
        heapq.heapify(temp_heap_s1)
        s1_selected = []
        used_s1 = set()
        
        while temp_heap_s1 and len(s1_selected) < 3:
            rem_neg, idx = heapq.heappop(temp_heap_s1)
            rem = -rem_neg
            if rem >= s1 and idx not in used_s1:
                s1_selected.append(idx)
                used_s1.add(idx)
        
        if len(s1_selected) < 3:
            return None
        
        for idx in s1_selected:
            folders[idx]['remaining'] -= s1
        
        temp_heap_s2 = [(-f['remaining'], f['index']) for f in folders]
        heapq.heapify(temp_heap_s2)
        s2_selected = []
        used_s2 = set()
        
        while temp_heap_s2 and len(s2_selected) < 3:
            rem_neg, idx = heapq.heappop(temp_heap_s2)
            rem = -rem_neg
            if rem >= s2 and idx not in used_s2:
                s2_selected.append(idx)
                used_s2.add(idx)
        
        if len(s2_selected) < 3:
            return None
        
        for idx in s2_selected:
            folders[idx]['remaining'] -= s2
        
        s1_locations = sorted(s1_selected)
        s2_locations = sorted(s2_selected)
        min_diff = float('inf')
        best_s2 = s2_locations
        
        for i in range(len(s2_locations) - 2):
            current = s2_locations[i:i+3]
            diff = sum(abs(current[j] - s1_locations[j]) for j in range(3))
            if diff < min_diff:
                min_diff = diff
                best_s2 = current
        
        allocations[file_id] = {
            'split': True,
            's1': {'size': s1, 'locations': s1_locations},
            's2': {'size': s2, 'locations': best_s2}
        }
    
    for f in folders:
        if f['remaining'] < 0:
            return None
    
    return allocations

def main():
    files = [1067, 799, 1585, 836, 944, 502, 1399, 783, 494, 712, 631, 1138, 434, 1107, 545, 1489] 
    files = [0.81 * _ for _ in files]
    result = allocate_files(files)
    print (result)
    
    if result is None:
        print("无法找到可行的分配策略。")
        return
    
    for file_id, details in result.items():
        if details['split']:
            print(f"文件 {file_id} 被拆分为两个部分:")
            print(f"  部分1大小 {details['s1']['size']}, 备份位置: {details['s1']['locations']}")
            print(f"  部分2大小 {details['s2']['size']}, 备份位置: {details['s2']['locations']}")
        else:
            print(f"文件 {file_id} 完整存储，备份位置: {details['locations']}")

if __name__ == "__main__":
    main()

None
无法找到可行的分配策略。


In [30]:
import itertools

def distribute_files(file_sizes, num_folders, folder_capacity):
    # 按文件大小降序排列以优化剪枝
    sorted_files = sorted(file_sizes, reverse=True)
    folder_sizes = [0] * num_folders
    best = {'max_files': 0, 'distribution': None}
    
    def backtrack(index, assigned):
        # 剪枝：剩余文件全分配也无法超过当前最优值时返回
        remaining = len(sorted_files) - index
        if assigned + remaining <= best['max_files']:
            return
        
        # 更新最优解
        if index == len(sorted_files):
            if assigned > best['max_files']:
                best['max_files'] = assigned
                best['distribution'] = folder_sizes.copy()
            return
        
        current_file = sorted_files[index]
        # 筛选可容纳当前文件的文件夹
        available = [f for f in range(num_folders) if folder_sizes[f] + current_file <= folder_capacity]
        
        # 若可用文件夹不足3个则跳过该文件
        if len(available) < 3:
            backtrack(index + 1, assigned)
            return
        
        # 按剩余容量降序排列以优先尝试更大空间的组合
        available.sort(key=lambda f: (folder_capacity - folder_sizes[f]), reverse=True)
        
        # 生成所有三文件夹组合
        for triplet in itertools.combinations(available, 3):
            # 分配文件到三个文件夹
            for f in triplet:
                folder_sizes[f] += current_file
            
            backtrack(index + 1, assigned + 1)  # 处理下一个文件
            
            # 回溯状态
            for f in triplet:
                folder_sizes[f] -= current_file
            
            # 提前终止条件：所有文件都已分配
            if best['max_files'] == len(sorted_files):
                return
        
        # 尝试跳过当前文件
        backtrack(index + 1, assigned)
    
    backtrack(0, 0)
    return best

# 输入参数
file_sizes = [1067, 799, 1585, 836, 944, 502, 1399, 783, 494, 712, 631, 1138, 434, 1107, 545, 1489]
num_folders = 10
folder_capacity = 5792

# 执行分配
result = distribute_files(file_sizes, num_folders, folder_capacity)

# 输出结果
print(f"最大可分配文件数: {result['max_files']}")
print(f"{result}")
print("各文件夹最终大小:")
for i, size in enumerate(result['distribution']):
    print(f"文件夹{i}: {size}")
sum(result['distribution'])

最大可分配文件数: 16
{'max_files': 16, 'distribution': [4431, 4412, 4404, 4418, 4420, 4404, 4048, 4048, 4407, 4403]}
各文件夹最终大小:
文件夹0: 4431
文件夹1: 4412
文件夹2: 4404
文件夹3: 4418
文件夹4: 4420
文件夹5: 4404
文件夹6: 4048
文件夹7: 4048
文件夹8: 4407
文件夹9: 4403


43395

In [ ]:
import time
from collections import deque

def allocate_files(file_sizes, N, V, timeout=5):
    start_time = time.time()
    folders = [{"capacity": V, "files": []} for _ in range(N)]
    file_queue = deque(sorted(enumerate(file_sizes), key=lambda x: -x[1]))
    split_files = set()
    original_to_parts = {}

    while file_queue and time.time() - start_time < timeout:
        file_id, size = file_queue.popleft()
        
        # 尝试分配原文件
        candidates = sorted([(i, f["capacity"]) for i, f in enumerate(folders)], 
                          key=lambda x: -x[1])
        available = [i for i, cap in candidates if cap >= size]
        
        if len(available) >= 3:
            selected = []
            used = set()
            for fidx in available:
                if len(selected) < 3 and fidx not in used:
                    selected.append(fidx)
                    used.add(fidx)
            if len(selected) == 3:
                for fidx in selected:
                    folders[fidx]["capacity"] -= size
                    folders[fidx]["files"].append(file_id)
                continue
        
        # 拆分文件
        split_size1 = size // 2
        split_size2 = size - split_size1
        split_files.add(file_id)
        
        # 生成拆分后的部分并加入队列
        part1 = (f"{file_id}_p1", split_size1)
        part2 = (f"{file_id}_p2", split_size2)
        file_queue.appendleft(part2)
        file_queue.appendleft(part1)
        original_to_parts[file_id] = [part1[0], part2[0]]

    # 处理最终分配结果
    allocation = []
    for folder in folders:
        valid_files = []
        for f in folder["files"]:
            if isinstance(f, int) and f not in split_files:
                valid_files.append(f)
            elif str(f).endswith(("_p1", "_p2")):
                original_id = int(f.split("_")[0]) # type: ignore
                if original_id in split_files:
                    valid_files.append(f)
        allocation.append(valid_files)

    return allocation, list(split_files)

# 输入参数
file_sizes = [1067, 799, 1585, 3333, 944, 502, 1399, 783, 494, 712, 
             631, 1138, 434, 1107, 545, 1489]
N = 10
V = 5792

# 执行分配
result, split_list = allocate_files(file_sizes, N, V)

print(result, split_list)
# 输出结果
for i, folder in enumerate(result):
    print(f"Folder {i}: {folder}")
print("\n被拆分的文件ID:", split_list)

sum_cost = [0 for _ in range(len(result))]
for i in range(len(result)):
    for j in range(len(result[0])):
        sum_cost[i] += file_sizes[result[i][j]]

print(sum_cost)

[[3, 1, 10, 12], [3, 1, 10, 12], [3, 1, 14, 8], [2, 11, 4, 7, 14], [2, 13, 0, 9, 5], [2, 13, 0, 9, 5], [15, 6, 4, 9, 8], [15, 6, 4, 10, 5, 12], [15, 11, 0, 7, 8], [6, 11, 13, 7, 14]] []
Folder 0: [3, 1, 10, 12]
Folder 1: [3, 1, 10, 12]
Folder 2: [3, 1, 14, 8]
Folder 3: [2, 11, 4, 7, 14]
Folder 4: [2, 13, 0, 9, 5]
Folder 5: [2, 13, 0, 9, 5]
Folder 6: [15, 6, 4, 9, 8]
Folder 7: [15, 6, 4, 10, 5, 12]
Folder 8: [15, 11, 0, 7, 8]
Folder 9: [6, 11, 13, 7, 14]

被拆分的文件ID: []
[5197, 5197, 5171, 4450, 4471, 4471, 4544, 4463, 4477, 4427]


: 

In [ ]:
obj_id = 0
MAX_REQUEST_NUM = MAX_OJECT_NUM = 0
index = []
position = []
timestamp = 0

In [ ]:

disk = [[obj_id for _ in range(V)] for _ in range(N)]  # obj_id: int
request_dict = [obj_id for _ in range(MAX_REQUEST_NUM)]  # obj_id: int

# index = [index1: int, index2: int, index3: int], position = [[position1], [position2], [position3]], position1: list[list[int]]
object_dict = [[index, position, size, timestamp] for _ in range(MAX_OJECT_NUM)]

In [11]:
sum(i> 0 for i in [-1, -1, 2, 2])

2

In [7]:
assign

[None,
 [None, 2, 5, 9, 12, 16],
 [None, 8, 10, 13, 15, 16],
 [None, 8, 10, 13, 15, 16],
 [None, 2, 3, 4, 5, 13],
 [None, 3, 6, 11, 12, 14],
 [None, 3, 6, 11, 12, 14],
 [None, 1, 7, 9, 11, 14],
 [None, 1, 2, 5, 7, 9],
 [None, 1, 4, 6, 8],
 [None, 4, 7, 10, 15]]